<a href="https://colab.research.google.com/github/danh21/AI_investigation/blob/main/back_propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
#load datasheet
print ('Load MNIST Database')
mnist = tf.keras.datasets.mnist
(x_train,y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train,(60000,784))/255.0
x_test = np.reshape(x_test,(10000,784))/255.0
y_train = np.matrix(np.eye(10)[y_train])
y_test = np.matrix(np.eye(10)[y_test])
print("------------------------------------------")
print(x_train.shape)
print(y_train.shape)

Load MNIST Database
------------------------------------------
(60000, 784)
(60000, 10)


In [ ]:
def sigmoid(x):
  return 1./(1.+np.exp(-x))

In [ ]:
def softmax(x):
  return np.divide(np.matrix(np.exp(x)),np.mat(np.sum(np.exp(x),axis=1)))

In [ ]:
def Forwardpass(X,Wh,bh,Wo,bo):
  zh = X@Wh.T + bh
  a = sigmoid(zh)
  z = a@Wo.T + bo
  o = softmax(z)
  return o

In [ ]:
def AccTest(label,prediction): #calculate the matching score
  OutMaxArg = np.argmax(prediction,axis=1)
  LabelMaxArg = np.argmax(label,axis=1)
  Accuracy = np.mean(OutMaxArg==LabelMaxArg)
  return Accuracy

In [ ]:
#define the network parameter
learningRate = 0.5
Epoch = 10
NumTrainSamples = 60000
NumTestSamples = 10000
NumInputs = 784
NumHiddenUnits = 512
NumClasses = 10

#initial weights
#hidden layer
Wh = np.matrix(np.random.uniform(-0.5,0.5,(NumHiddenUnits,NumInputs)))
bh = np.random.uniform(0,0.5,(1,NumHiddenUnits))
dWh = np.zeros((NumHiddenUnits,NumInputs))
dbh = np.zeros((1,NumHiddenUnits))
#Output layer
Wo = np.random.uniform(-0.5,0.5,(NumClasses,NumHiddenUnits))
bo = np.random.uniform(0,0.5,(1,NumClasses))
dWo = np.zeros((NumClasses,NumHiddenUnits))
dbo = np.zeros((1,NumClasses))

In [ ]:
#Test accuracy with random initial weights
prediction = Forwardpass(x_train,Wh,bh,Wo,bo)
Acc = AccTest(y_train,prediction)
print(Acc)

0.09675


In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
loss = []
Acc = []
for ep in range (Epoch):
  #feed fordware propagation
  x = x_train
  y = y_train
  zh = x@Wh.T + bh
  a = sigmoid(zh)
  z = a@Wo.T + bo
  o = softmax(z)
  #calculate loss
  loss.append(-np.sum(np.multiply(y,np.log10(o))))
  #calculate the error for the output layer
  d = o - y
  #back propagate error
  dh = d@Wo
  dhs = np.multiply(np.multiply(dh,a),(1-a))
  #update weight
  dWo = np.matmul(np.transpose(d),a)
  dbo = np.mean(d) #consider a is 1 for bias
  dWh = np.matmul(np.transpose(dhs),x)
  dbh = np.mean(dhs) #consider a is 1 for bias
  Wo = Wo - learningRate * dWo/NumTrainSamples
  bo = bo - learningRate * dbo
  Wh = Wh - learningRate * dWh/NumTrainSamples
  bh = bh - learningRate * dbh
  #Test accuracy with random initial weights
  prediction = Forwardpass(x_test,Wh,bh,Wo,bo)
  Acc.append(AccTest(y_test,prediction))
  clear_output(wait=True)
  plt.plot([i for i, _ in enumerate(Acc)],Acc,'o')
  plt.show() #càng ngày càng chính xác hơn

In [ ]:
#Test accuracy with random initial weights
prediction = Forwardpass(x_train,Wh,bh,Wo,bo)
Acc = AccTest(y_train,prediction)
print(Acc)

0.4963166666666667


In [ ]:
#mini-batch SGD
from IPython.display import clear_output
import matplotlib.pyplot as plt
loss = []
Acc = []
Batch_size = 200
Stochastic_samples = np.arange(NumTrainSamples) # 0,1,2,3,4,5,6,7,...,59999
for ep in range (Epoch):
  np.random.shuffle(Stochastic_samples)
  for ite in range (0,NumTrainSamples,Batch_size):
    #feed fordware propagation
    Batch_samples = Stochastic_samples[ite:ite+Batch_size]
    x = x_train[Batch_samples,:]
    y = y_train[Batch_samples,:]
    zh = x@Wh.T + bh
    a = sigmoid(zh)
    z = a@Wo.T + bo
    o = softmax(z)
    #calculate loss
    loss.append(-np.sum(np.multiply(y,np.log10(o))))
    #calculate the error for the output layer
    d = o - y
    #back propagate error
    dh = d@Wo
    dhs = np.multiply(np.multiply(dh,a),(1-a))
    #update weight
    dWo = np.matmul(np.transpose(d),a)
    dbo = np.mean(d) #consider a is 1 for bias
    dWh = np.matmul(np.transpose(dhs),x)
    dbh = np.mean(dhs) #consider a is 1 for bias
    Wo = Wo - learningRate * dWo/Batch_size
    bo = bo - learningRate * dbo
    Wh = Wh - learningRate * dWh/Batch_size
    bh = bh - learningRate * dbh
  #Test accuracy with random initial weights
  prediction = Forwardpass(x_test,Wh,bh,Wo,bo)
  Acc.append(AccTest(y_test,prediction))
  print('Epoch:',ep)
  print('Accuracy:',AccTest(y_test,prediction))

Epoch: 0
Accuracy: 0.9002
Epoch: 1
Accuracy: 0.9352
Epoch: 2
Accuracy: 0.9434
Epoch: 3
Accuracy: 0.9471
Epoch: 4
Accuracy: 0.9529
Epoch: 5
Accuracy: 0.9555
Epoch: 6
Accuracy: 0.9582
Epoch: 7
Accuracy: 0.9582
Epoch: 8
Accuracy: 0.9631
Epoch: 9
Accuracy: 0.9629
